In [9]:
from kafka import KafkaConsumer, KafkaProducer
import json
from jsonschema import validate, ValidationError

# Załaduj schemat z pliku
with open('sensor_schema.json') as f:
    schema = json.load(f)

consumer = KafkaConsumer(
    'raw_iot',
    bootstrap_servers='kafka_streaming_lab:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='raw-validator',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

producer = KafkaProducer(
    bootstrap_servers='kafka_streaming_lab:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

print("Validator started...")

with open("error_log.log", "w") as logfile:
    for msg in consumer:
        data = msg.value
        try:
            validate(instance=data, schema=schema)
            producer.send('iot_data', value=data)
            print(f"Valid message: {data}")
        except ValidationError as e:
            error_record = {
                "device_id": data.get("device_id"),
                "error_message": "temperature or humidity not in accepted range",
                "event_timestamp": data.get("timestamp")
            }
            producer.send('error_log', value=error_record)
            print(f"Invalid message: {data}")
            logfile.write(f"{json.dumps(data)}\n")


Validator started...
Invalid message: {'device_id': 'sensor-000', 'temperature': 5.02, 'humidity': 168.28, 'timestamp': 1748018108.9132106}
Invalid message: {'device_id': 'sensor-002', 'temperature': 64.64, 'humidity': 145.39, 'timestamp': 1748018109.914909}
Invalid message: {'device_id': 'sensor-002', 'temperature': 73.21, 'humidity': 105.04, 'timestamp': 1748018111.4162714}
Invalid message: {'device_id': 'sensor-002', 'temperature': 76.72, 'humidity': 72.97, 'timestamp': 1748018112.9174767}
Invalid message: {'device_id': 'sensor-002', 'temperature': -26.25, 'humidity': 107.05, 'timestamp': 1748018114.4194136}
Invalid message: {'device_id': 'sensor-001', 'temperature': -18.5, 'humidity': 9.13, 'timestamp': 1748018115.4203558}
Invalid message: {'device_id': 'sensor-002', 'temperature': 96.98, 'humidity': 24.59, 'timestamp': 1748018117.4229424}
Invalid message: {'device_id': 'sensor-000', 'temperature': 11.4, 'humidity': 188.26, 'timestamp': 1748018117.923822}
Invalid message: {'device_

KeyboardInterrupt: 